In [ ]:

import torch.nn as nn
import json
import os
import enum

# Visualization related imports
import matplotlib.pyplot as plt
import networkx as nx
from networkx.readwrite import json_graph

# Main computation libraries
import numpy as np

# Deep learning related imports
import torch
from torch.utils.data import DataLoader, Dataset
class GATLayer(torch.nn.Module):
    """
    Implementation #3 was inspired by PyTorch Geometric: https://github.com/rusty1s/pytorch_geometric

    But, it's hopefully much more readable! (and of similar performance)

    """
    
    # We'll use these constants in many functions so just extracting them here as member fields
    src_nodes_dim = 0  # position of source nodes in edge index
    trg_nodes_dim = 1  # position of target nodes in edge index

    # These may change in the inductive setting - leaving it like this for now (not future proof)
    nodes_dim = 0      # node dimension (axis is maybe a more familiar term nodes_dim is the position of "N" in tensor)
    head_dim = 1       # attention head dim

    def __init__(self, num_in_features, num_out_features, num_of_heads, concat=True, activation=nn.ELU(),
                 dropout_prob=0.6, add_skip_connection=True, bias=True, log_attention_weights=False):

        super().__init__()

        self.num_of_heads = num_of_heads
        self.num_out_features = num_out_features
        self.concat = concat  # whether we should concatenate or average the attention heads
        self.add_skip_connection = add_skip_connection

        #
        # Trainable weights: linear projection matrix (denoted as "W" in the paper), attention target/source
        # (denoted as "a" in the paper) and bias (not mentioned in the paper but present in the official GAT repo)
        #

        # You can treat this one matrix as num_of_heads independent W matrices
        self.linear_proj = nn.Linear(num_in_features, num_of_heads * num_out_features, bias=False)

        # After we concatenate target node (node i) and source node (node j) we apply the "additive" scoring function
        # which gives us un-normalized score "e". Here we split the "a" vector - but the semantics remain the same.
        # Basically instead of doing [x, y] (concatenation, x/y are node feature vectors) and dot product with "a"
        # we instead do a dot product between x and "a_left" and y and "a_right" and we sum them up
        self.scoring_fn_target = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))
        self.scoring_fn_source = nn.Parameter(torch.Tensor(1, num_of_heads, num_out_features))

        # Bias is definitely not crucial to GAT - feel free to experiment (I pinged the main author, Petar, on this one)
        if bias and concat:
            self.bias = nn.Parameter(torch.Tensor(num_of_heads * num_out_features))
        elif bias and not concat:
            self.bias = nn.Parameter(torch.Tensor(num_out_features))
        else:
            self.register_parameter('bias', None)

        if add_skip_connection:
            self.skip_proj = nn.Linear(num_in_features, num_of_heads * num_out_features, bias=False)
        else:
            self.register_parameter('skip_proj', None)

        #
        # End of trainable weights
        #

        self.leakyReLU = nn.LeakyReLU(0.2)  # using 0.2 as in the paper, no need to expose every setting
        self.activation = activation
        # Probably not the nicest design but I use the same module in 3 locations, before/after features projection
        # and for attention coefficients. Functionality-wise it's the same as using independent modules.
        self.dropout = nn.Dropout(p=dropout_prob)

        self.log_attention_weights = log_attention_weights  # whether we should log the attention weights
        self.attention_weights = None  # for later visualization purposes, I cache the weights here

        self.init_params()
        
    def forward(self, data):
        #
        # Step 1: Linear Projection + regularization
        #
        in_nodes_features, edge_index = data  # unpack data
        num_of_nodes = in_nodes_features.shape[self.nodes_dim]
        assert edge_index.shape[0] == 2, f'Expected edge index with shape=(2,E) got {edge_index.shape}'

        # shape = (N, FIN) where N - number of nodes in the graph, FIN - number of input features per node
        # We apply the dropout to all of the input node features (as mentioned in the paper)
        in_nodes_features = self.dropout(in_nodes_features)

        # shape = (N, FIN) * (FIN, NH*FOUT) -> (N, NH, FOUT) where NH - number of heads, FOUT - num of output features
        # We project the input node features into NH independent output features (one for each attention head)
        nodes_features_proj = self.linear_proj(in_nodes_features).view(-1, self.num_of_heads, self.num_out_features)

        nodes_features_proj = self.dropout(nodes_features_proj)  # in the official GAT imp they did dropout here as well

        #
        # Step 2: Edge attention calculation
        #

        # Apply the scoring function (* represents element-wise (a.k.a. Hadamard) product)
        # shape = (N, NH, FOUT) * (1, NH, FOUT) -> (N, NH, 1) -> (N, NH) because sum squeezes the last dimension
        # Optimization note: torch.sum() is as performant as .sum() in my experiments
        scores_source = (nodes_features_proj * self.scoring_fn_source).sum(dim=-1)
        scores_target = (nodes_features_proj * self.scoring_fn_target).sum(dim=-1)

        # We simply copy (lift) the scores for source/target nodes based on the edge index. Instead of preparing all
        # the possible combinations of scores we just prepare those that will actually be used and those are defined
        # by the edge index.
        # scores shape = (E, NH), nodes_features_proj_lifted shape = (E, NH, FOUT), E - number of edges in the graph
        scores_source_lifted, scores_target_lifted, nodes_features_proj_lifted = self.lift(scores_source, scores_target, nodes_features_proj, edge_index)
        scores_per_edge = self.leakyReLU(scores_source_lifted + scores_target_lifted)

        # shape = (E, NH, 1)
        attentions_per_edge = self.neighborhood_aware_softmax(scores_per_edge, edge_index[self.trg_nodes_dim], num_of_nodes)
        # Add stochasticity to neighborhood aggregation
        attentions_per_edge = self.dropout(attentions_per_edge)

        #
        # Step 3: Neighborhood aggregation
        #

        # Element-wise (aka Hadamard) product. Operator * does the same thing as torch.mul
        # shape = (E, NH, FOUT) * (E, NH, 1) -> (E, NH, FOUT), 1 gets broadcast into FOUT
        nodes_features_proj_lifted_weighted = nodes_features_proj_lifted * attentions_per_edge

        # This part sums up weighted and projected neighborhood feature vectors for every target node
        # shape = (N, NH, FOUT)
        out_nodes_features = self.aggregate_neighbors(nodes_features_proj_lifted_weighted, edge_index, in_nodes_features, num_of_nodes)

        #
        # Step 4: Residual/skip connections, concat and bias
        #

        out_nodes_features = self.skip_concat_bias(attentions_per_edge, in_nodes_features, out_nodes_features)
        return (out_nodes_features, edge_index)

    #
    # Helper functions (without comments there is very little code so don't be scared!)
    #

    def neighborhood_aware_softmax(self, scores_per_edge, trg_index, num_of_nodes):
        """
        As the fn name suggest it does softmax over the neighborhoods. Example: say we have 5 nodes in a graph.
        Two of them 1, 2 are connected to node 3. If we want to calculate the representation for node 3 we should take
        into account feature vectors of 1, 2 and 3 itself. Since we have scores for edges 1-3, 2-3 and 3-3
        in scores_per_edge variable, this function will calculate attention scores like this: 1-3/(1-3+2-3+3-3)
        (where 1-3 is overloaded notation it represents the edge 1-3 and its (exp) score) and similarly for 2-3 and 3-3
         i.e. for this neighborhood we don't care about other edge scores that include nodes 4 and 5.

        Note:
        Subtracting the max value from logits doesn't change the end result but it improves the numerical stability
        and it's a fairly common "trick" used in pretty much every deep learning framework.
        Check out this link for more details:

        https://stats.stackexchange.com/questions/338285/how-does-the-subtraction-of-the-logit-maximum-improve-learning

        """
        # Calculate the numerator. Make logits <= 0 so that e^logit <= 1 (this will improve the numerical stability)
        scores_per_edge = scores_per_edge - scores_per_edge.max()
        exp_scores_per_edge = scores_per_edge.exp()  # softmax

        # Calculate the denominator. shape = (E, NH)
        neigborhood_aware_denominator = self.sum_edge_scores_neighborhood_aware(exp_scores_per_edge, trg_index, num_of_nodes)

        # 1e-16 is theoretically not needed but is only there for numerical stability (avoid div by 0) - due to the
        # possibility of the computer rounding a very small number all the way to 0.
        attentions_per_edge = exp_scores_per_edge / (neigborhood_aware_denominator + 1e-16)

        # shape = (E, NH) -> (E, NH, 1) so that we can do element-wise multiplication with projected node features
        return attentions_per_edge.unsqueeze(-1)

    def sum_edge_scores_neighborhood_aware(self, exp_scores_per_edge, trg_index, num_of_nodes):
        # The shape must be the same as in exp_scores_per_edge (required by scatter_add_) i.e. from E -> (E, NH)
        trg_index_broadcasted = self.explicit_broadcast(trg_index, exp_scores_per_edge).long()

        # shape = (N, NH), where N is the number of nodes and NH the number of attention heads
        size = list(exp_scores_per_edge.shape)  # convert to list otherwise assignment is not possible
        size[self.nodes_dim] = num_of_nodes
        neighborhood_sums = torch.zeros(size, dtype=exp_scores_per_edge.dtype, device=exp_scores_per_edge.device)

        # position i will contain a sum of exp scores of all the nodes that point to the node i (as dictated by the
        # target index)
        neighborhood_sums.scatter_add_(self.nodes_dim, trg_index_broadcasted, exp_scores_per_edge)

        # Expand again so that we can use it as a softmax denominator. e.g. node i's sum will be copied to
        # all the locations where the source nodes pointed to i (as dictated by the target index)
        # shape = (N, NH) -> (E, NH)
        return neighborhood_sums.index_select(self.nodes_dim, trg_index.long())

    def aggregate_neighbors(self, nodes_features_proj_lifted_weighted, edge_index, in_nodes_features, num_of_nodes):
        size = list(nodes_features_proj_lifted_weighted.shape)  # convert to list otherwise assignment is not possible
        size[self.nodes_dim] = num_of_nodes  # shape = (N, NH, FOUT)
        out_nodes_features = torch.zeros(size, dtype=in_nodes_features.dtype, device=in_nodes_features.device)

        # shape = (E) -> (E, NH, FOUT)
        trg_index_broadcasted = self.explicit_broadcast(edge_index[self.trg_nodes_dim], nodes_features_proj_lifted_weighted).long()
        # aggregation step - we accumulate projected, weighted node features for all the attention heads
        # shape = (E, NH, FOUT) -> (N, NH, FOUT)
        out_nodes_features.scatter_add_(self.nodes_dim, trg_index_broadcasted, nodes_features_proj_lifted_weighted)

        return out_nodes_features

    def lift(self, scores_source, scores_target, nodes_features_matrix_proj, edge_index):
        """
        Lifts i.e. duplicates certain vectors depending on the edge index.
        One of the tensor dims goes from N -> E (that's where the "lift" comes from).

        """
        src_nodes_index = edge_index[self.src_nodes_dim].long()
        trg_nodes_index = edge_index[self.trg_nodes_dim].long()

        # Using index_select is faster than "normal" indexing (scores_source[src_nodes_index]) in PyTorch!
        scores_source = scores_source.index_select(self.nodes_dim, src_nodes_index)
        scores_target = scores_target.index_select(self.nodes_dim, trg_nodes_index)
        nodes_features_matrix_proj_lifted = nodes_features_matrix_proj.index_select(self.nodes_dim, src_nodes_index)

        return scores_source, scores_target, nodes_features_matrix_proj_lifted

    def explicit_broadcast(self, this, other):
        # Append singleton dimensions until this.dim() == other.dim()
        for _ in range(this.dim(), other.dim()):
            this = this.unsqueeze(-1)

        # Explicitly expand so that shapes are the same
        return this.expand_as(other)

    def init_params(self):
        """
        The reason we're using Glorot (aka Xavier uniform) initialization is because it's a default TF initialization:
            https://stackoverflow.com/questions/37350131/what-is-the-default-variable-initializer-in-tensorflow

        The original repo was developed in TensorFlow (TF) and they used the default initialization.
        Feel free to experiment - there may be better initializations depending on your problem.

        """
        nn.init.xavier_uniform_(self.linear_proj.weight)
        nn.init.xavier_uniform_(self.scoring_fn_target)
        nn.init.xavier_uniform_(self.scoring_fn_source)

        if self.bias is not None:
            torch.nn.init.zeros_(self.bias)

    def skip_concat_bias(self, attention_coefficients, in_nodes_features, out_nodes_features):
        if self.log_attention_weights:  # potentially log for later visualization in playground.py
            self.attention_weights = attention_coefficients

        if self.add_skip_connection:  # add skip or residual connection
            if out_nodes_features.shape[-1] == in_nodes_features.shape[-1]:  # if FIN == FOUT
                # unsqueeze does this: (N, FIN) -> (N, 1, FIN), out features are (N, NH, FOUT) so 1 gets broadcast to NH
                # thus we're basically copying input vectors NH times and adding to processed vectors
                out_nodes_features += in_nodes_features.unsqueeze(1)
            else:
                # FIN != FOUT so we need to project input feature vectors into dimension that can be added to output
                # feature vectors. skip_proj adds lots of additional capacity which may cause overfitting.
                out_nodes_features += self.skip_proj(in_nodes_features).view(-1, self.num_of_heads, self.num_out_features)
        if self.concat:
            # shape = (N, NH, FOUT) -> (N, NH*FOUT)
            out_nodes_features = out_nodes_features.view(-1, self.num_of_heads * self.num_out_features)
        else:
            # shape = (N, NH, FOUT) -> (N, FOUT)
            out_nodes_features = out_nodes_features.mean(dim=self.head_dim)

        if self.bias is not None:
            out_nodes_features += self.bias

        return out_nodes_features if self.activation is None else self.activation(out_nodes_features)
class VAE(nn.Module):

    def __init__(self,latent_size,node_num,num_features,num_of_layers, num_heads_per_layer, num_features_per_layer, add_skip_connection=True, bias=True,
                 dropout=0.6, log_attention_weights=False):
        
        super().__init__()
        num_heads_per_layer = [1] + num_heads_per_layer
        self.latent_size = latent_size

        self.encoder = Encoder(
            latent_size,node_num,num_features,num_of_layers, num_heads_per_layer, num_features_per_layer, add_skip_connection, bias,
                 dropout, log_attention_weights)
        self.decoder = Decoder(
            latent_size,node_num,num_features,num_of_layers, num_heads_per_layer, num_features_per_layer, add_skip_connection, bias,
                 dropout, log_attention_weights)

    def forward(self, x,edge_index):
        x = (x,edge_index)
        means, log_var = self.encoder(x)
        z = self.sample_from_gaussian(means, log_var)
        means_x, log_var_x = self.decoder(z,edge_index)
        x = self.sample_from_gaussian(means_x, log_var_x)
        return means_x,log_var_x, means, log_var,z,x


    
    def reconstruction_probability(self,x,edge_index):
        self.encoder.eval()
        self.decoder.eval()
        enc = (x,edge_index)
        mu, logvar = self.encoder(enc)
        x = x.view(-1)
        z_samples = self.samples_from_gaussian(mu,logvar)
#         likelyhoods = torch.tensor(float(0))
        inum = 0
        loss = 0
        old_zb = 0
        for ind , z_sample in enumerate(z_samples):
            x_mean, x_logvar = self.decoder(z_sample,edge_index)
            log_recon_likelihood = -0.5 * (torch.sum(torch.pow(x-x_mean,2) * torch.exp(-x_logvar) , axis = [0]) + torch.sum(x_logvar ,axis=0) + args.node_num * np.log(2*np.pi))
            T = log_recon_likelihood
            kl_divergence = -0.5 * torch.sum(1 + logvar - torch.pow(mu,2) - torch.exp(logvar), axis=0)
            if ind == 0:
                loss = torch.mean(kl_divergence - log_recon_likelihood)
                old_zb = T
            else:
                loss +=torch.mean(kl_divergence - log_recon_likelihood)
                old_zb += T

        
        return loss/10,old_zb/10
    def reparameterize(self, mu, log_var):

        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)

        return mu + eps * std
                                     
    def sample_from_gaussian(self, mean,logvar):
        eps = torch.randn_like(mean)
        return eps.mul(torch.exp(logvar/2)).add_(mean)
    
    def samples_from_gaussian(self, mean, logvar, T=10):
        z_samples = []
        for _ in range(T):
            eps = torch.randn_like(mean)
            z_samples.append(eps.mul(torch.exp(logvar/2)).add_(mean))
        return z_samples
    def inference(self, z):

        recon_x = self.decoder(z)

        return recon_x


class Encoder(nn.Module):

    def __init__(self,latent_size,node_num,num_features,num_of_layers, num_heads_per_layer, num_features_per_layer, add_skip_connection, bias,
                 dropout, log_attention_weights):

        super().__init__()
        
        gat_layers = []  # collect GAT layers
        for i in range(num_of_layers):
            layer = GATLayer(
                num_in_features=num_features_per_layer[i] * num_heads_per_layer[i],  # consequence of concatenation
                num_out_features=num_features_per_layer[i+1],
                num_of_heads=num_heads_per_layer[i+1],
                concat=True if i < num_of_layers - 1 else False,  # last GAT layer does mean avg, the others do concat
                activation=nn.ELU() if i < num_of_layers - 1 else None,  # last layer just outputs raw scores
                dropout_prob=dropout,
                add_skip_connection=add_skip_connection,
                bias=bias,
                log_attention_weights=log_attention_weights
            )
            gat_layers.append(layer)

        self.MLP = nn.Sequential(
            *gat_layers,
        )
        self.linear_means = nn.Linear(num_features*node_num, latent_size)
        self.linear_log_var = nn.Linear(num_features*node_num, latent_size)

    def forward(self, x):
        x = self.MLP(x)[0]
        x = torch.flatten(x, start_dim=1)
        x = torch.squeeze(x)
        means = self.linear_means(x)
        log_vars = self.linear_log_var(x)

        return means, log_vars


class Decoder(nn.Module):

    def __init__(self,latent_size,node_num,num_features,num_of_layers, num_heads_per_layer, num_features_per_layer, add_skip_connection, bias,
                 dropout, log_attention_weights):

        super().__init__()
        self.node_num = node_num
        self.num_features = num_features
        self.decoder_input = nn.Linear(latent_size,node_num*num_features)
        gat_layers = []  # collect GAT layers
        for i in range(num_of_layers):
            layer = GATLayer(
                num_in_features=num_features_per_layer[i] * num_heads_per_layer[i],  # consequence of concatenation
                num_out_features=num_features_per_layer[i+1],
                num_of_heads=num_heads_per_layer[i+1],
                concat=True if i < num_of_layers - 1 else False,  # last GAT layer does mean avg, the others do concat
                activation=nn.ELU() if i < num_of_layers - 1 else None,  # last layer just outputs raw scores
                dropout_prob=dropout,
                add_skip_connection=add_skip_connection,
                bias=bias,
                log_attention_weights=log_attention_weights
            )
            gat_layers.append(layer)

        self.MLP = nn.Sequential(
            *gat_layers,
        )
        self.linear_means = nn.Linear(num_features*node_num, num_features*node_num)
        self.linear_log_var = nn.Linear(num_features*node_num, num_features*node_num)
    def forward(self, z,edge_index):
        z = self.decoder_input(z)
        z = z.view(-1,self.node_num,self.num_features)
        z = torch.squeeze(z, dim=0)
        z = (z,edge_index)
        x = self.MLP(z)[0]
        x = torch.flatten(x, start_dim=1)
        x = torch.squeeze(x)
        means = self.linear_means(x)
        log_vars = self.linear_log_var(x)
        return means,log_vars

import torch.utils.data as data
class MyDataset(data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __getitem__(self, index):#返回的是tensor
        img, target = self.images[index], self.labels[index]
        return img, target

    def __len__(self):
        return len(self.images)


def load_graph_data(edgepath,featpath):
    edge_index_list = []
    node_features_list = []
    file=open(edgepath,"rb")
    edge_list = pickle.load(file)
    
    dfnp = np.array(edge_list).reshape((-1,args.node_num,args.node_num))
    dfnp = dfnp + np.identity(args.node_num)
    for i in range(len(dfnp)):
        size = np.sum(dfnp[i]==1)
        midnp = np.zeros((2,size))
        sign = 0
        for j in range(args.node_num):
            for t in range(args.node_num):
                if dfnp[i][j][t] ==1:
                    midnp[0][sign] =j
                    midnp[1][sign] =t
                    sign+=1
        edge_index_list.append(midnp)

    fun = np.loadtxt(open(featpath,"rb"), delimiter=",", skiprows=0)
    funnp = fun.reshape((-1,args.node_num,1))
    
    
    def get_mean_std(matrix):
        mean = []
        std = []
        for item in np.transpose(matrix):
            mean.append(np.mean(item[item>0.00001]))
            std.append(max(1, np.std(item[item>0.00001])))
        return mean, std
    def normalization(matrix, mean, std):
        n_mat = np.array(matrix, dtype=np.float32)
        n_mat = np.where(n_mat<0.00001, 0, (n_mat - mean) / std)
        return n_mat

    
    funnp = funnp.reshape((-1,args.node_num))
    mean,std = get_mean_std(funnp)
    funnp = normalization(funnp, mean, std)
    funnp = funnp.reshape((-1,args.node_num,1))
    return (funnp, edge_index_list),mean,std


def load_val_data(edgepath,featpath,mean,std):
    valfeatureslist = np.loadtxt(open(featpath,"rb"), delimiter=",", skiprows=0)
    file=open(edgepath,"rb")
    valedge_list = pickle.load(file)
    
    valedge_list = np.array(valedge_list).reshape((-1,args.node_num,args.node_num))
    valfeatureslist = valfeatureslist.reshape((-1,args.node_num,1))

    valedge_list = valedge_list + np.identity(args.node_num)

    val_edge_index_list = []
    for i in range(len(valedge_list)):
        size = np.sum(valedge_list[i]==1)
        midnp = np.zeros((2,size))
        sign = 0
        for j in range(args.node_num):
            for t in range(args.node_num):
                if valedge_list[i][j][t] ==1:
                    midnp[0][sign] =j
                    midnp[1][sign] =t
                    sign+=1
        val_edge_index_list.append(midnp)
    
        

    def normalization(matrix, mean, std):
        n_mat = np.array(matrix, dtype=np.float32)
        n_mat = np.where(n_mat<0.00001, 0, (n_mat - mean) / std)
        return n_mat
    
    
    valfeatureslist = valfeatureslist.reshape((-1,args.node_num))
    valfeatureslist = normalization(valfeatureslist, mean, std)
    valfeatureslist = valfeatureslist.reshape((-1,args.node_num,1))
    
    return (valfeatureslist, val_edge_index_list)
        
        
        
import os
import time
import torch
import argparse
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from collections import defaultdict
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def main(args):
    trainfeatpath = ''
    trainedgepath = ''
    ooo = 0
    torch.manual_seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(args.seed)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #显存较大的显卡可以使用batch拼接实现批次计算，显存不足可以采用单图计算
    data_loader,m,n = load_graph_data(trainedgepath,trainfeatpath)
    dataset = MyDataset(data_loader[0], data_loader[1])
    data_loader = DataLoader(
        dataset=dataset, batch_size=1, shuffle=True)
    
    #valfeatpath = ''
    #valedgepath = ''
    # val_data_loader = load_val_data(valedgepath,valfeatpath,m,n)
    # datasetval = MyDataset(val_data_loader[0], val_data_loader[1])
    # data_loaderval = DataLoader(
    #     dataset=datasetval, batch_size=1, shuffle=True)
    

    
            
    vae = VAE(
        node_num=args.node_num,
        latent_size=args.latent_size,
        num_features=args.num_features,
        num_of_layers=args.num_of_layers,
        num_heads_per_layer=args.num_heads_per_layer,
        num_features_per_layer=args.num_features_per_layer,
        dropout=args.dropout).to(device)

    optimizer = torch.optim.Adam(vae.parameters(), lr=args.learning_rate)
#     early_stopping = EarlyStopping(patience=10, verbose=False, path=f'/home/parameter/{args.experiment}.pth')
    
    logs = defaultdict(list)
    firloss = 9999
    inum = 0
    lastloss = 9999
    for epoch in range(args.epochs):
        indexnum = 1
        loss = 0
        epochloss = 0
        lossmm = 0
        
        for iteration, (node_features, edge_index) in enumerate(data_loader):
            node_features = torch.squeeze(node_features, dim=0)
            edge_index = torch.squeeze(edge_index)
            node_features = torch.tensor(node_features, dtype=torch.float32)
            edge_index = torch.tensor(edge_index, dtype=torch.int64)
            edge_index = edge_index.to(device)
            node_features = node_features.to(device)
            
            mean_x,log_var_x ,mean, log_var,z,recon_x = vae(node_features,edge_index)

            kl_divergence = -0.5 * torch.sum(1 + log_var - torch.pow(mean,2) - torch.exp(log_var), axis=0)
            log_recon_likelihood = -0.5 * (torch.sum(torch.pow(node_features.reshape((args.node_num,))-mean_x.reshape((args.node_num,)),2) * torch.exp(-log_var_x.reshape((args.node_num,))) , axis = [0]) + torch.sum(log_var_x.reshape((args.node_num,)) ,axis=0) + args.node_num * np.log(2*np.pi))

            
            likeout = torch.mean(log_recon_likelihood)
            nloss = torch.mean(kl_divergence - log_recon_likelihood)


            lossmm+=nloss
            if indexnum ==1:
                loss = nloss
                epochloss += nloss
            else:
                loss += nloss
                epochloss += nloss
            if indexnum % args.batch_size ==0:
                loss = loss/args.batch_size 
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss = 0
            if indexnum == args.train_lenght:
                loss = loss/(args.train_lenght - int(args.train_lenght/args.batch_size)*args.batch_size)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss = 0
            indexnum+=1
            if indexnum % 1000 ==0:
                #print(f'阶段性loss:{lossmm/1000}')
                lossmm =0
        print(f'\033[1;35;46m 第{epoch+1}轮 loss:   {epochloss/args.train_lenght} \033[0m')


In [ ]:
import pickle
parser = argparse.ArgumentParser()
parser.add_argument("--seed", type=int, default=0)
parser.add_argument("--epochs", type=int, default=300)
parser.add_argument("--batch_size", type=int, default=256)
parser.add_argument("--learning_rate", type=float, default=0.001)
parser.add_argument("--node_num", type=int, default=100)
parser.add_argument("--num_features", type=list, default=1)
parser.add_argument("--latent_size", type=int, default=10)
parser.add_argument("--print_every", type=int, default=100)
parser.add_argument("--fig_root", type=str, default='figs')
parser.add_argument("--num_of_layers", type=int, default=3)
parser.add_argument("--num_heads_per_layer", type=list, default=[4, 6 , 4])
parser.add_argument("--num_features_per_layer", type=list, default=[1,6,4,1])
parser.add_argument("--dropout", type=float, default=0)
parser.add_argument("--trace_type", type=int, default=40)
parser.add_argument("--train_lenght", type=int, default=10000)
parser.add_argument('--experiment', type=str, default='Abnormal_class_GAT_vae', help='experiment name')
args = parser.parse_known_args()[0]
#main(args)


import os
import time
import torch
import argparse
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from collections import defaultdict
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vae = VAE(
        node_num=args.node_num,
        latent_size=args.latent_size,
        num_features=args.num_features,
        num_of_layers=args.num_of_layers,
        num_heads_per_layer=args.num_heads_per_layer,
        num_features_per_layer=args.num_features_per_layer,
        dropout=0).to(device)
#加载训练好的模型
parameter = torch.load(os.path.join(''))
vae.load_state_dict(parameter)

In [ ]:
#X1为数据集特征矩阵地址 E1为邻接矩阵地址，T1为调用链类型存储地址，L1为调用信息
#nodelistxx为数据集中所有故障文件名称列表
X1 = ""
E1 = ""
T1 = ""
L1 = ""
nodelistxx = [] 
def load_test_data1(mean,std):
    nodenumxx = [0]
    feattest = np.zeros((1,args.node_num,1))
    edgetest = []
    typely = np.zeros(1)
    nodelistname = []
    edgell = []
    feattt = np.zeros((1,args.node_num,1))
    for nodenamexx in nodelistxx : 
        X1x = X1.split("|")[0] + nodenamexx +X1.split("|")[1]
        E1e = E1.split("|")[0] + nodenamexx +E1.split("|")[1]
        T1t = T1.split("|")[0] + nodenamexx +T1.split("|")[1]
        L1l = L1.split("|")[0] + nodenamexx +L1.split("|")[1]
        
        abnormal = np.loadtxt(open(X1x,"rb"), delimiter=",", skiprows=0)
        file=open(E1e,"rb")
        abnormalexg = pickle.load(file)
        ynum2 = np.loadtxt(open(T1t,"rb"), delimiter=",", skiprows=0)
        file=open(L1l,"rb")
        yy2 = pickle.load(file)
        edgell.extend(abnormalexg)
        abnormalexg = np.array(abnormalexg).reshape((-1,args.node_num,args.node_num))
        abnormal = abnormal.reshape((-1,args.node_num,1))
        feattt = np.concatenate((feattt, abnormal), axis=0)
        abnormalexg = abnormalexg + np.identity(args.node_num)
        test_edge_index_list = []
        for i in range(len(abnormalexg)):
            size = np.sum(abnormalexg[i]==1)
            midnp = np.zeros((2,size))
            sign = 0
            for j in range(args.node_num):
                for t in range(args.node_num):
                    if abnormalexg[i][j][t] ==1:
                        midnp[0][sign] =j
                        midnp[1][sign] =t
                        sign+=1
            test_edge_index_list.append(midnp)
        
        nodenumxx.append(len(abnormal)+nodenumxx[len(nodenumxx)-1])
        feattest = np.concatenate((feattest, abnormal), axis=0)
        typely = np.concatenate((typely, ynum2), axis=0)
        nodelistname.extend(yy2)
        edgetest.extend(test_edge_index_list)
        
    def normalization(matrix, mean, std):
        n_mat = np.array(matrix, dtype=np.float32)
        n_mat = np.where(n_mat<0.00001, 0, (n_mat - mean) / std)
        return n_mat
    
    feattest = np.delete(feattest, 0, axis=0)
    typely = np.delete(typely, 0, axis=0)
    feattt = np.delete(feattt, 0, axis=0)
    graphfunp = feattest.reshape((-1,args.node_num))
    graphfunp = normalization(graphfunp, mean, std)
    graphfunp = graphfunp.reshape((-1,args.node_num,1))
    
    return (graphfunp, edgetest,nodelistname,typely,nodenumxx,edgell,feattt)
class MyDataset1(data.Dataset):
    def __init__(self, images, labels,y,num_y):
        self.images = images
        self.labels = labels
        self.y = y
        self.num_y = num_y
    def __getitem__(self, index):#返回的是tensor
        img, target ,y,num_y= self.images[index], self.labels[index],self.y[index],self.num_y[index]
        return img, target,y,num_y

    def __len__(self):
        return len(self.images)


import numpy as np

data_loader,m,n = load_graph_data()

test_loader = load_test_data1(m,n)

nodenumxx = test_loader[4]
start_time = time.time() 
testset = MyDataset1(test_loader[0], test_loader[1],test_loader[2],test_loader[3])
test_loader1 = DataLoader(dataset=testset, batch_size=1, shuffle=False)

ablikely = []
yy = []
num_ylist = []
itrll = 0
for iteration, (node_features, edge_index,y,num_y) in enumerate(test_loader1):
    node_features = torch.squeeze(node_features, dim=0)
    edge_index = torch.squeeze(edge_index)
    node_features = torch.tensor(node_features, dtype=torch.float32)
    edge_index = torch.tensor(edge_index, dtype=torch.int64)
    edge_index = edge_index.to(device)
    node_features = node_features.to(device)
    loss,old_zb= vae.reconstruction_probability(node_features,edge_index)
    yy.append(y)
    ablikely.append(old_zb)
    num_ylist.append(num_y)

In [ ]:
#加载分类阈值和正常值
midvalue = np.loadtxt(open('',"rb"), delimiter=",", skiprows=0)
file=open(r"","rb")
midmod = pickle.load(file)
midvalue = midvalue
nodenumx = [0]
abnormalexg = np.zeros((1,args.node_num,1))
trainid = np.zeros(1)
nodenamelist = []
abnormal = np.zeros((1,args.node_num,args.node_num))
for ix in range(1,len(nodelistxx)+1): 
    abnormal1 = test_loader[6][nodenumxx[ix-1] : nodenumxx[ix]]
    abnormalexg1 = np.array(test_loader[5]).reshape(-1,args.node_num,args.node_num)[nodenumxx[ix-1] : nodenumxx[ix]]
    yy2 = test_loader[2][nodenumxx[ix-1] : nodenumxx[ix]]
    ynum2 = test_loader[3][nodenumxx[ix-1] : nodenumxx[ix]]

    ablike = np.array(torch.tensor(ablikely, device='cpu'))[nodenumxx[ix-1] : nodenumxx[ix]]
    abnumlist = np.array(torch.tensor(num_ylist, device='cpu'))[nodenumxx[ix-1] : nodenumxx[ix]]
    typelt = ynum2
    num = []
    for i in range(len(ablike)):
        if ablike[i] < midvalue[int(abnumlist[i])]/1:
            num.append(i)
    nodenumx.append(len(num)+nodenumx[len(nodenumx)-1])
    abnormalexg = np.concatenate((abnormalexg, abnormal1[num]), axis=0)
    trainid = np.concatenate((trainid, ynum2[num]), axis=0)
    nodenamelist.extend([yy2[iyt] for iyt in num])
    abnormal = np.concatenate((abnormal, abnormalexg1[num].reshape(-1,args.node_num,args.node_num)), axis=0)

abnormalexg = np.delete(abnormalexg, 0, axis=0)
trainid = np.delete(trainid, 0, axis=0)   
abnormal = np.delete(abnormal, 0, axis=0)
edge = abnormal.reshape(-1,args.node_num,args.node_num)

In [1]:
from queue import LifoQueue
def onlynode(abnormalnum, headnum, abormalfeatfun,tid):
    weightindex = np.argwhere(edge[abnormalnum][headnum]==1)
    weightnum = len(weightindex)
    weighttree = np.repeat(abormalfeatfun.reshape((1,args.node_num)), repeats=1, axis=0)
    j = 0
    allexcess = 0
    for i in range(weightnum):
        excess = 0
        excess += weighttree[j][weightindex[i][0]] - midmod[tid][weightindex[i][0]]
        weighttree[j][weightindex[i][0]] = midmod[tid][weightindex[i][0]]
        q=LifoQueue()
        index = np.argwhere(edge[abnormalnum][weightindex[i][0]]==1)
        for indexnum in range(len(index)):
            q.put(index[indexnum][0])
        while not q.empty():
            qget = q.get()
            weighttree[j][qget] = midmod[tid][qget]
            index = np.argwhere(edge[abnormalnum][qget]==1)
            for indexnum in range(len(index)):
                q.put(index[indexnum][0])
        allexcess+=excess
        weighttree[j][headnum] = weighttree[j][headnum]-excess
        index = np.argwhere(edge[abnormalnum][:,headnum]==1)
        for indexnum in range(len(index)):
            q.put(index[indexnum][0])
        while not q.empty():
            qget = q.get()
            weighttree[j][qget] = weighttree[j][qget]-excess
            index = np.argwhere(edge[abnormalnum][:,qget]==1)
            for indexnum in range(len(index)):
                q.put(index[indexnum][0])
    return weighttree,allexcess
def manynode(abnormalnum, headnum, abormalfeatfun,tid):
    weightindex = np.argwhere(edge[abnormalnum][headnum]==1)
    weightnum = len(weightindex)
    weighttree = np.repeat(abormalfeatfun.reshape((1,args.node_num)), repeats=weightnum, axis=0)
    from queue import LifoQueue
    for i in range(weightnum):
        excess = 0
        new = np.delete(weightindex, i, axis=0)
        for j in range(len(new)):
            excess += weighttree[i][new[j][0]] - midmod[tid][new[j][0]]
            weighttree[i][new[j][0]] = midmod[tid][new[j][0]]
            q=LifoQueue()
            index = np.argwhere(edge[abnormalnum][new[j][0]]==1)
            for indexnum in range(len(index)):
                q.put(index[indexnum][0])
            while not q.empty():
                qget = q.get()
                weighttree[i][qget] = midmod[tid][qget]
                index = np.argwhere(edge[abnormalnum][qget]==1)
                for indexnum in range(len(index)):
                    q.put(index[indexnum][0])
        weighttree[i][headnum] = weighttree[i][headnum]-excess
        index = np.argwhere(edge[abnormalnum][:,headnum]==1)
        for indexnum in range(len(index)):
            q.put(index[indexnum][0])
        while not q.empty():
            qget = q.get()
            weighttree[i][qget] = weighttree[i][qget]-excess
            index = np.argwhere(edge[abnormalnum][:,qget]==1)
            for indexnum in range(len(index)):
                q.put(index[indexnum][0]) 
    return weighttree,weightindex
def modelpd(abnormalnum, weighttree, m,n):
    edge_index_list = []
    dfnp = abnormal[abnormalnum].reshape((-1,args.node_num,args.node_num))
    dfnp = dfnp + np.identity(args.node_num)
    for i in range(len(dfnp)):
        size = np.sum(dfnp[i]==1)
        midnp = np.zeros((2,size))
        sign = 0
        for j in range(args.node_num):
            for t in range(args.node_num):
                if dfnp[i][j][t] ==1:
                    midnp[0][sign] =j
                    midnp[1][sign] =t
                    sign+=1
        edge_index_list.append(midnp)

    def normalization(matrix, mean, std):
            n_mat = np.array(matrix, dtype=np.float32)
            n_mat = np.where(n_mat<0.00001, 0, (n_mat - mean) / std)
            return n_mat

    graphfunp = weighttree.reshape((-1,args.node_num))
    graphfunp = normalization(graphfunp, m, n)
    graphfunp = graphfunp.reshape((-1,args.node_num,1))
    like = []
    with torch.no_grad():
        for iteration in range(len(graphfunp)):
            node_features = torch.tensor(graphfunp[iteration], dtype=torch.float32)
            edge_index = torch.tensor(edge_index_list[0], dtype=torch.int64)
            node_features = torch.squeeze(node_features, dim=0)
            edge_index = torch.squeeze(edge_index)
            edge_index = edge_index.to(device)
            node_features = node_features.to(device)
            loss,old_zb= vae.reconstruction_probability(node_features,edge_index)
            like.append(old_zb)
            like_tensor = torch.tensor(like)
            like_cpu = like_tensor.cpu()
            likely = np.array(like_cpu)
    return likely

In [ ]:
abnodeall = []
import queue
import time
from tqdm import tqdm
abnormalvalue = 1
abnormalnodeall = []
abnormallikeall = []
for abnormalnum in tqdm(range(abnormalexg.shape[0])):
    tid = int(trainid[abnormalnum])
    abnormalnode = []
    abnormallike = []
    abormal0 = abnormalexg[abnormalnum].copy()
    headnum = tid
    weighttree,allexcess = onlynode(abnormalnum, headnum, abormal0,tid)
    likely = modelpd(abnormalnum, weighttree, m,n)
    if likely<midvalue[tid]/abnormalvalue:
        abnormalnode.append(headnum)
        ablike = []
        ablike.append(headnum)
        ablike.append(likely)
        abnormallike.append(ablike)
        abormal0[headnum] = midmod[tid][headnum]+allexcess
    Q=queue.Queue()
    index = np.argwhere(edge[abnormalnum][headnum]==1)
    if len(index)==1:
        headfeat = []
        headfeat.append(index[0])
        headfeat.append(abormal0)
        headfeat.append(0)
        Q.put(headfeat)
    if len(index)>1:
        headfeat = []
        headfeat.append(tid)
        headfeat.append(abormal0)
        headfeat.append(1)
        Q.put(headfeat)
    while not Q.empty():
        headfeat = Q.get()
        if headfeat[2]==0:
            weighttree,allexcess = onlynode(abnormalnum, headfeat[0][0], headfeat[1],tid)
            likely = modelpd(abnormalnum, weighttree, m,n)
            if likely<midvalue[tid]/abnormalvalue:
                abnormalnode.append(headfeat[0][0])
                ablike = []
                ablike.append(headfeat[0][0])
                ablike.append(likely)
                abnormallike.append(ablike)
                abormalcess = headfeat[1][headfeat[0][0]]-allexcess-midmod[tid][headfeat[0][0]]
                headfeat[1][headfeat[0][0]] = allexcess+midmod[tid][headfeat[0][0]]
                index = np.argwhere(edge[abnormalnum][:,headfeat[0][0]]==1)
                q=LifoQueue()
                for indexnum in range(len(index)):
                    q.put(index[indexnum][0])
                while not q.empty():
                    qget = q.get()
                    headfeat[1][qget] = headfeat[1][qget]-abormalcess
                    index = np.argwhere(edge[abnormalnum][:,qget]==1)
                    for indexnum in range(len(index)):
                        q.put(index[indexnum][0])
            index = np.argwhere(edge[abnormalnum][headfeat[0][0]]==1)
            if len(index)==1:
                headfeat1 = []
                headfeat1.append(index[0])
                headfeat1.append(headfeat[1])
                headfeat1.append(0)
                Q.put(headfeat1)
            if len(index)>1:
                headfeat1 = []
                headfeat1.append(headfeat[0][0])
                headfeat1.append(headfeat[1])
                headfeat1.append(1)
                Q.put(headfeat1)
        if headfeat[2]==1:
            weighttree,weightindex = manynode(abnormalnum, headfeat[0], headfeat[1],tid)
            likely = modelpd(abnormalnum, weighttree, m,n)
            abnormalindex = np.argwhere(likely<midvalue[tid]/abnormalvalue)
            for indexnum in range(len(abnormalindex)):
                headfeat1 = []
                headfeat1.append(weightindex[abnormalindex[indexnum]][0])
                headfeat1.append(weighttree[abnormalindex[indexnum]][0])
                headfeat1.append(0)
                Q.put(headfeat1)
    def takeSecond(elem):
        return elem[1]
#     指定第二个元素排序
    abnormallike.sort(key=takeSecond)
    abnormalnodeall.append(abnormalnode)
    abnormallikeall.append(abnormallike)

In [ ]:
abnodeall = []
for ix in range(1,len(nodelistxx)+1):
    abno = abnormalnodeall[nodenumx[ix-1]:nodenumx[ix]]
    abnoli = nodenamelist[nodenumx[ix-1]:nodenumx[ix]]
    abnoo = []
    for ite in range(len(abno)):
        for i in abno[ite]:
            if i <args.trace_type:
                abnoo.append(abnoli[ite][0])
            else:
                abnoo.append(abnoli[ite][i-(args.trace_type-1)])
    abnodeall.append(abnoo)
            
import math
def returnSum(myDict):   
    sum = 0
    for i in myDict: 
        sum = sum + myDict[i] 
    return sum

yy4 = test_loader[2]

top1 = 0
top3 = 0
top5 = 0
nul = 0


for ix in range(1,len(nodelistxx)+1):
    abnode = abnodeall[ix-1]
    nodedict = dict()
    blyy = yy4[nodenumxx[ix-1]:nodenumxx[ix]]
    for i in range(len(blyy)):
        for noden in blyy[i]:
            if "->" in noden:
                nodesz = noden.split("->")
                nodesz[0] = "net-"+nodesz[0]
                nodesz[1] = "net-"+nodesz[1]
                if nodesz[0] in nodedict.keys():
                    nodedict[nodesz[0]] = nodedict[nodesz[0]]+1
                else:
                    nodedict[nodesz[0]] = 1
                if nodesz[1] in nodedict.keys():
                    nodedict[nodesz[1]] = nodedict[nodesz[1]]+1
                else:
                    nodedict[nodesz[1]] = 1
                continue
            if noden in nodedict.keys():
                nodedict[noden] = nodedict[noden]+1
            else:
                nodedict[noden] = 1
    abnodedict = dict()
    for noden in abnode:
        if "->" in noden:
            nodesz = noden.split("->")
            nodesz[0] = "net-"+nodesz[0]
            nodesz[1] = "net-"+nodesz[1]
            if nodesz[0] in abnodedict.keys():
                abnodedict[nodesz[0]] = abnodedict[nodesz[0]]+1
            else:
                abnodedict[nodesz[0]] = 1
            if nodesz[1] in abnodedict.keys():
                abnodedict[nodesz[1]] = abnodedict[nodesz[1]]+1
            else:
                abnodedict[nodesz[1]] = 1
            continue
        if noden in abnodedict.keys():
            abnodedict[noden] = abnodedict[noden]+1
        else:
            abnodedict[noden] = 1
    abnodedict

    scdict = dict()
    for node in abnodedict.keys():
        scdict[node] = abnodedict[node]/nodedict[node]
        oef = abnodedict[node]
        oep = nodedict[node] - abnodedict[node]
        onf = returnSum(abnodedict) - abnodedict[node]
        onp = returnSum(nodedict) - nodedict[node]
        scdict[node] = oef/math.sqrt((oef+oep)*(oef+onf))
    
    score = dict()
    for node in scdict.keys():
        name = node.split("@")[0]
        if "net" in name:
            name = name[4:]
        name = str(name)
        if name in score.keys():
            score[name] = score[name] + scdict[node]
        else:
            score[name] = scdict[node]
    
    

    print(f"------------------{nodelistxx[ix-1]}-----------------------------------------")
    retdict = sorted(score.items(), key=lambda x: -x[1])
    namelistxl = str(nodelistxx[ix-1])
    print(retdict)
    scdict.clear()
    nodedict.clear()
    abnodedict.clear()
    score.clear()
